In [1]:
import tensorflow as tf 
import keras 
import keras.backend as K
from scipy.signal import resample

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from collections import Counter

from keras import regularizers
from keras.models import Sequential, Model, load_model, model_from_json 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Reshape, Concatenate,  Dropout 
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}



class Estimator:
    l2p = 0.001
    @staticmethod
    def early_layers(inp, fm = (1,3), hid_act_func="relu"):
        # Start
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 1
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)

        return x
    
    @staticmethod
    def late_layers(inp, num_classes, fm = (1,3), act_func="softmax", hid_act_func="relu", b_name="Identifier"):
        # 2
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        
        # End
        x = Flatten()(x)
        x = Dense(128, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(32, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes, activation=act_func, name = b_name)(x)

        return x
   
    @staticmethod
    def build(height, width, num_classes, name, fm = (1,3), act_func="softmax",hid_act_func="relu"):
        inp = Input(shape=(height, width, 1))
        early = Estimator.early_layers(inp, fm, hid_act_func=hid_act_func)
        late  = Estimator.late_layers(early, num_classes, fm, act_func=act_func, hid_act_func=hid_act_func)
        model = Model(inputs=inp, outputs=late ,name=name)
        return model

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt


def get_ds_infos():
    """
    Read the file includes data subject information.
    
    Data Columns:
    0: code [1-24]
    1: weight [kg]
    2: height [cm]
    3: age [years]
    4: gender [0:Female, 1:Male]
    
    Returns:
        A pandas DataFrame that contains inforamtion about data subjects' attributes 
    """ 

    dss = pd.read_csv("data_subjects_info.csv")
    print("[INFO] -- Data subjects' information is imported.")
    
    return dss

def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and the mode to shape the final dataset.
    
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration] 

    Returns:
        It returns a list of columns to use for creating time-series from files.
    """
    dt_list = []
    for t in data_types:
        if t != "attitude":
            dt_list.append([t+".x",t+".y",t+".z"])
        else:
            dt_list.append([t+".roll", t+".pitch", t+".yaw"])

    return dt_list


def creat_time_series(dt_list, act_labels, trial_codes, mode="mag", labeled=True, combine_grav_acc=False):
    """
    Args:
        dt_list: A list of columns that shows the type of data we want.
        act_labels: list of activites
        trial_codes: list of trials
        mode: It can be "raw" which means you want raw data
        for every dimention of each data type,
        [attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)].
        or it can be "mag" which means you only want the magnitude for each data type: (x^2+y^2+z^2)^(1/2)
        labeled: True, if we want a labeld dataset. False, if we only want sensor values.
        combine_grav_acc: True, means adding each axis of gravity to  corresponding axis of userAcceleration.
    Returns: 
        It returns a time-series of sensor data.
    
    """
    num_data_cols = len(dt_list) if mode == "mag" else len(dt_list*3)

    if labeled:
        dataset = np.zeros((0,num_data_cols+7)) # "7" --> [act, code, weight, height, age, gender, trial] 
    else:
        dataset = np.zeros((0,num_data_cols))
        
    ds_list = get_ds_infos()
    
    print("[INFO] -- Creating Time-Series")
    for sub_id in ds_list["code"]:
        for act_id, act in enumerate(act_labels):
            for trial in trial_codes[act_id]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                vals = np.zeros((len(raw_data), num_data_cols))
                
                if combine_grav_acc:
                    raw_data["userAcceleration.x"] = raw_data["userAcceleration.x"].add(raw_data["gravity.x"])
                    raw_data["userAcceleration.y"] = raw_data["userAcceleration.y"].add(raw_data["gravity.y"])
                    raw_data["userAcceleration.z"] = raw_data["userAcceleration.z"].add(raw_data["gravity.z"])
                
                for x_id, axes in enumerate(dt_list):
                    if mode == "mag":
                        vals[:,x_id] = (raw_data[axes]**2).sum(axis=1)**0.5        
                    else:
                        vals[:,x_id*3:(x_id+1)*3] = raw_data[axes].values
                    vals = vals[:,:num_data_cols]
                if labeled:
                    lbls = np.array([[act_id,
                            sub_id-1,
                            ds_list["weight"][sub_id-1],
                            ds_list["height"][sub_id-1],
                            ds_list["age"][sub_id-1],
                            ds_list["gender"][sub_id-1],
                            trial          
                           ]]*len(raw_data))
                    vals = np.concatenate((vals, lbls), axis=1)
                dataset = np.append(dataset,vals, axis=0)
    cols = []
    for axes in dt_list:
        if mode == "raw":
            cols += axes
        else:
            cols += [str(axes[0][:-2])]
            
    if labeled:
        cols += ["act", "id", "weight", "height", "age", "gender", "trial"]
    
    dataset = pd.DataFrame(data=dataset, columns=cols)
    return dataset
#________________________________
#________________________________

def ts_to_secs(dataset, w, s, standardize = False, **options):
    
    data = dataset[dataset.columns[:-7]].values    
    act_labels = dataset["act"].values
    id_labels = dataset["id"].values
    trial_labels = dataset["trial"].values

    mean = 0
    std = 1
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
            print("[INFO] -- Test Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("[INFO] -- Training Data has been standardized: the mean is = "+str(mean)+" ; and the std is = "+str(std))            

        data -= mean
        data /= std
    else:
        print("[INFO] -- Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T

    m = data.shape[0]   # Data Dimension 
    ttp = data.shape[1] # Total Time Points
    number_of_secs = int(round(((ttp - w)/s)))

    ##  Create a 3D matrix for Storing Sections  
    secs_data = np.zeros((number_of_secs , m , w ))
    act_secs_labels = np.zeros(number_of_secs)
    id_secs_labels = np.zeros(number_of_secs)

    k=0
    for i in range(0 , ttp-w, s):
        j = i // s
        if j >= number_of_secs:
            break
        if id_labels[i] != id_labels[i+w-1]: 
            continue
        if act_labels[i] != act_labels[i+w-1]: 
            continue
        if trial_labels[i] != trial_labels[i+w-1]:
            continue
            
        secs_data[k] = data[:, i:i+w]
        act_secs_labels[k] = act_labels[i].astype(int)
        id_secs_labels[k] = id_labels[i].astype(int)
        k = k+1
        
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    id_secs_labels = id_secs_labels[0:k]
    return secs_data, act_secs_labels, id_secs_labels, mean, std
##________________________________________________________________


ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11],
    ACT_LABELS[1]:[3,4,12],
    ACT_LABELS[2]:[7,8,15],
    ACT_LABELS[3]:[9,16],
    ACT_LABELS[4]:[6,14],
    ACT_LABELS[5]:[5,13],
}

In [3]:
#https://stackoverflow.com/a/45305384/5210098
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [4]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
results ={}
sdt = ["rotationRate","userAcceleration"]
mode = "mag"
cga = True # Add gravity to acceleration or not

print("[INFO] -- Selected sensor data types: "+str(sdt)+" -- Mode: "+str(mode)+" -- Grav+Acc: "+str(cga))    
act_labels = ACT_LABELS [0:4]

print("[INFO] -- Selected activites: "+str(act_labels))    
trial_codes = [TRIAL_CODES[act] for act in act_labels]
dt_list = set_data_types(sdt)
dataset = creat_time_series(dt_list, act_labels, trial_codes, mode=mode, labeled=True, combine_grav_acc = cga)
print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    


#*****************
TRAIN_TEST_TYPE = "trial" # "subject" or "trial"
#*****************

if TRAIN_TEST_TYPE == "subject":
    test_ids = [4,9,11,21]
    print("[INFO] -- Test IDs: "+str(test_ids))
    test_ts = dataset.loc[(dataset['id'].isin(test_ids))]
    train_ts = dataset.loc[~(dataset['id'].isin(test_ids))]
else:
    test_trail = [11,12,13,14,15,16]  
    print("[INFO] -- Test Trials: "+str(test_trail))
    test_ts = dataset.loc[(dataset['trial'].isin(test_trail))]
    train_ts = dataset.loc[~(dataset['trial'].isin(test_trail))]

print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
print("[INFO] -- Shape of Test Time-Series :"+str(test_ts.shape))

# print("___________Train_VAL____________")
# val_trail = [11,12,13,14,15,16]
# val_ts = train_ts.loc[(train_ts['trial'].isin(val_trail))]
# train_ts = train_ts.loc[~(train_ts['trial'].isin(val_trail))]
# print("[INFO] -- Training Time-Series :"+str(train_ts.shape))
# print("[INFO] -- Validation Time-Series :"+str(val_ts.shape))

print("___________________________________________________")
print(train_ts.head())

## This Variable Defines the Size of Sliding Window
## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
w = 128 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
s = 10
train_data, act_train, id_train, train_mean, train_std = ts_to_secs(train_ts.copy(),
                                                                   w,
                                                                   s,
                                                                   standardize = True)

s = 10
# val_data, act_val, id_val, val_mean, val_std = ts_to_secs(val_ts.copy(),
#                                                           w,
#                                                           s,
#                                                           standardize = True,
#                                                           mean = train_mean, 
#                                                           std = train_std)

s = 10
test_data, act_test, id_test, test_mean, test_std = ts_to_secs(test_ts.copy(),
                                                              w,
                                                              s,
                                                              standardize = True,
                                                              mean = train_mean, 
                                                              std = train_std)

print("[INFO] -- Training Sections: "+str(train_data.shape))
#print("[INFO] -- Validation Sections: "+str(val_data.shape))
print("[INFO] -- Test Sections:  "+str(test_data.shape))


id_train_labels = to_categorical(id_train)
#id_val_labels = to_categorical(id_val)
id_test_labels = to_categorical(id_test)

act_train_labels = to_categorical(act_train)
#act_val_labels = to_categorical(act_val)
act_test_labels = to_categorical(act_test)

[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 9)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 9)
[INFO] -- Shape of Test Time-Series :(145687, 9)
___________________________________________________
   rotationRate  userAcceleration  act   id  weight  height   age  gender  \
0      1.370498          1.195847  0.0  0.0   102.0   188.0  46.0     1.0   
1      1.141648          1.196990  0.0  0.0   102.0   188.0  46.0     1.0   
2      0.372530          1.117437  0.0  0.0   102.0   188.0  46.0     1.0   
3      1.049628          1.088320  0.0  0.0   102.0   188.0  46.0     1.0   
4      0.921229          1.390551  0.0  0.0   102.0   188.0  46.0     1.0   

   trial  
0    1.0  
1    1.0  
2    1.0

In [5]:
train_data, act_train_labels, id_train_labels = shuffle(train_data, act_train_labels, id_train_labels, random_state=0)

In [6]:
bck_train_data = train_data.copy()
#bck_val_data = val_data.copy()
bck_test_data = test_data.copy()

In [7]:
def eval_act_id(sdt, mode, ep, cga, num_sampels):
    global bck_train_data, bck_test_data
    global act_train_labels,  act_test_labels
    ## DownSampling ##
    if  num_sampels!= 128:
        ds_train_data = bck_train_data.copy()
        #ds_val_data = bck_val_data.copy()
        ds_test_data = bck_test_data.copy()
    
        for sens in range(2):
            tmp = np.array([resample(x,num_sampels) for x in ds_train_data[:,sens,:]])
            ds_train_data[:,sens,:num_sampels] = tmp
            
#             tmp = np.array([resample(x,num_sampels) for x in ds_val_data[:,sens,:]])
#             ds_val_data[:,sens,:num_sampels] = tmp

            tmp = np.array([resample(x,num_sampels) for x in ds_test_data[:,sens,:]])
            ds_test_data[:,sens,:num_sampels] = tmp 

        ds_train_data = ds_train_data[:,:,:num_sampels]
       # ds_val_data = ds_val_data[:,:,:num_sampels]
        ds_test_data = ds_test_data[:,:,:num_sampels]

        print("[INFO] -- Training Sections:", ds_train_data.shape)
        #print("[INFO] -- Validation Sections:", ds_val_data.shape)
        print("[INFO] -- Test Sections:", ds_test_data.shape)

        train_data = ds_train_data
        #val_data = ds_val_data
        test_data = ds_test_data
        width = num_sampels
    else:
        train_data = bck_train_data.copy()
        #val_data = bck_val_data.copy()
        test_data = bck_test_data.copy()

    height = train_data.shape[1]
    width = train_data.shape[2]
    ## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
    train_data = np.expand_dims(train_data,axis=3)
    print("[INFO] -- Training Sections:", train_data.shape)
#     val_data = np.expand_dims(val_data,axis=3)
#     print("[INFO] -- Validation Sections:", val_data.shape)
    test_data = np.expand_dims(test_data,axis=3)
    print("[INFO] -- Test Sections:", test_data.shape)

   

    height = train_data.shape[1]
    width = train_data.shape[2]

    id_class_numbers = 24
    act_class_numbers = 4
    fm = (2,5)

    print("___________________________________________________")
    ## Callbacks
    eval_metric= "val_acc"    
    #eval_metric= "val_f1_metric"    
    early_stop = keras.callbacks.EarlyStopping(monitor=eval_metric, mode='max', patience = 7)
    filepath="XXACT.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint, early_stop]
    ## Callbacks

    eval_act = Estimator.build(height, width, id_class_numbers, name ="EVAL_ACT", fm=fm, act_func="softmax",hid_act_func="relu")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])
    print("Model Size = "+str(eval_act.count_params()))

    eval_act.fit(train_data, id_train_labels,
                validation_split = .2,
                epochs = ep,
                batch_size = 128,
                verbose = 0,
                class_weight = get_class_weights(np.argmax(id_train_labels,axis=1)),
                callbacks = callbacks_list
               )

    eval_act.load_weights("XXACT.best.hdf5")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])

    result1 = eval_act.evaluate(test_data, id_test_labels, verbose = 2)
    act_acc = result1[1].round(4)*100
    print("***[RESULT]*** ID Accuracy: "+str(act_acc))

    preds = eval_act.predict(test_data)
    preds = np.argmax(preds, axis=1)
    conf_mat = confusion_matrix(np.argmax(id_test_labels, axis=1), preds)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print("***[RESULT]*** ID  Confusion Matrix")
    print((np.array(conf_mat).diagonal()).round(3)*100)  

    f1act = f1_score(np.argmax(id_test_labels, axis=1), preds, average=None).mean()
    print("***[RESULT]*** ID Averaged F-1 Score : "+str(f1act))

    return f1act

In [8]:
ep = 32

In [10]:
#*******************
sample_rate = 50 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 50 --> Number of Samples: 128
[INFO] -- Training Sections: (60059, 2, 128, 1)
[INFO] -- Test Sections: (13344, 2, 128, 1)
___________________________________________________
Model Size = 158040

Epoch 00001: val_acc improved from -inf to 0.33758, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.33758 to 0.46329, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.46329 to 0.65693, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.65693 to 0.80411, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.80411 to 0.85331, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.85331 to 0.88745, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.88745 to 0.91425, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.91425 to 0.92949, saving model to XXACT.best.hdf5

Epoch 00009: val_acc improved from 0.92949 to 0.93815, saving model to XXACT.

In [11]:
#*******************
sample_rate = 45 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 45 --> Number of Samples: 115
[INFO] -- Training Sections: (60059, 2, 115)
[INFO] -- Test Sections: (13344, 2, 115)
[INFO] -- Training Sections: (60059, 2, 115, 1)
[INFO] -- Test Sections: (13344, 2, 115, 1)
___________________________________________________
Model Size = 141656

Epoch 00001: val_acc improved from -inf to 0.35206, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.35206 to 0.54837, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.54837 to 0.65534, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.65534 to 0.73685, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.73685 to 0.81652, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.81652 to 0.88811, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.88811 to 0.91700, saving model to XXACT.best.hdf5

Epoch 00008: val_acc did not improve from 0.91700

Epoch 00009: val_acc did not

In [9]:
#*******************
sample_rate = 40 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 40 --> Number of Samples: 102
[INFO] -- Training Sections: (60059, 2, 102)
[INFO] -- Test Sections: (13344, 2, 102)
[INFO] -- Training Sections: (60059, 2, 102, 1)
[INFO] -- Test Sections: (13344, 2, 102, 1)
___________________________________________________
Model Size = 125272

Epoch 00001: val_acc improved from -inf to 0.32118, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.32118 to 0.54129, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.54129 to 0.68648, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.68648 to 0.77314, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.77314 to 0.83400, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.83400 to 0.86547, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.86547 to 0.88820, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.88820 to 0.92840, saving model to XXACT.be

In [10]:
#*******************
sample_rate = 35 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 35 --> Number of Samples: 89
[INFO] -- Training Sections: (60059, 2, 89)
[INFO] -- Test Sections: (13344, 2, 89)
[INFO] -- Training Sections: (60059, 2, 89, 1)
[INFO] -- Test Sections: (13344, 2, 89, 1)
___________________________________________________
Model Size = 117080

Epoch 00001: val_acc improved from -inf to 0.35856, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.35856 to 0.51557, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.51557 to 0.68806, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.68806 to 0.71337, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.71337 to 0.82209, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.82209 to 0.84574, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.84574 to 0.90301, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.90301 to 0.91217, saving model to XXACT.best.hd

In [9]:
#*******************
sample_rate = 30 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 30 --> Number of Samples: 76
[INFO] -- Training Sections: (60059, 2, 76)
[INFO] -- Test Sections: (13344, 2, 76)
[INFO] -- Training Sections: (60059, 2, 76, 1)
[INFO] -- Test Sections: (13344, 2, 76, 1)
___________________________________________________
Model Size = 100696

Epoch 00001: val_acc improved from -inf to 0.39411, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.39411 to 0.56460, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.56460 to 0.65493, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.65493 to 0.75508, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.75508 to 0.84008, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.84008 to 0.85556, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.85556 to 0.89269, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.89269 to 0.91642, saving model to XXACT.best.hd

In [9]:
#*******************
sample_rate = 25 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] --  For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] --  For Sample Rate: 25 --> Number of Samples: 64
[INFO] -- Training Sections: (60059, 2, 64)
[INFO] -- Test Sections: (13344, 2, 64)
[INFO] -- Training Sections: (60059, 2, 64, 1)
[INFO] -- Test Sections: (13344, 2, 64, 1)
___________________________________________________
Model Size = 92504

Epoch 00001: val_acc improved from -inf to 0.29196, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.29196 to 0.49093, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.49093 to 0.66300, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.66300 to 0.80977, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.80977 to 0.85614, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.85614 to 0.89236, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.89236 to 0.89427, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.89427 to 0.93390, saving model to XXACT.best.hd

In [9]:
#*******************
sample_rate = 20 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 20 --> Number of Samples: 51
[INFO] -- Training Sections: (60059, 2, 51)
[INFO] -- Test Sections: (13344, 2, 51)
[INFO] -- Training Sections: (60059, 2, 51, 1)
[INFO] -- Test Sections: (13344, 2, 51, 1)
___________________________________________________
Model Size = 76120

Epoch 00001: val_acc improved from -inf to 0.38953, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.38953 to 0.53854, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.53854 to 0.66175, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.66175 to 0.80428, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.80428 to 0.83608, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.83608 to 0.86605, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.86605 to 0.89427, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.89427 to 0.91009, saving model to XXACT.best.hdf

In [10]:
#*******************
sample_rate = 15 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 15 --> Number of Samples: 38
[INFO] -- Training Sections: (60059, 2, 38)
[INFO] -- Test Sections: (13344, 2, 38)
[INFO] -- Training Sections: (60059, 2, 38, 1)
[INFO] -- Test Sections: (13344, 2, 38, 1)
___________________________________________________
Model Size = 59736

Epoch 00001: val_acc improved from -inf to 0.36630, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.36630 to 0.48077, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.48077 to 0.64594, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.64594 to 0.74451, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.74451 to 0.76474, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.76474 to 0.82676, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.82676 to 0.84657, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.84657 to 0.87746, saving model to XXACT.best.hdf

In [11]:
#*******************
sample_rate = 10 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 10 --> Number of Samples: 25
[INFO] -- Training Sections: (60059, 2, 25)
[INFO] -- Test Sections: (13344, 2, 25)
[INFO] -- Training Sections: (60059, 2, 25, 1)
[INFO] -- Test Sections: (13344, 2, 25, 1)
___________________________________________________
Model Size = 51544

Epoch 00001: val_acc improved from -inf to 0.38553, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.38553 to 0.50450, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.50450 to 0.60306, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.60306 to 0.65759, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.65759 to 0.72752, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.72752 to 0.78080, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.78080 to 0.79396, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.79396 to 0.81926, saving model to XXACT.best.hdf

In [9]:
#*******************
sample_rate = 5 #Hz
#*******************
num_sampels = (128*sample_rate)//50
print("[INFO] -- For Sample Rate: "+str(sample_rate)+" --> Number of Samples: "+str(num_sampels))
results[sample_rate] = eval_act_id(sdt, mode, ep, cga, num_sampels)

[INFO] -- For Sample Rate: 5 --> Number of Samples: 12
[INFO] -- Training Sections: (60059, 2, 12)
[INFO] -- Test Sections: (13344, 2, 12)
[INFO] -- Training Sections: (60059, 2, 12, 1)
[INFO] -- Test Sections: (13344, 2, 12, 1)
___________________________________________________
Model Size = 35160

Epoch 00001: val_acc improved from -inf to 0.27189, saving model to XXACT.best.hdf5

Epoch 00002: val_acc improved from 0.27189 to 0.42666, saving model to XXACT.best.hdf5

Epoch 00003: val_acc improved from 0.42666 to 0.47852, saving model to XXACT.best.hdf5

Epoch 00004: val_acc improved from 0.47852 to 0.52239, saving model to XXACT.best.hdf5

Epoch 00005: val_acc improved from 0.52239 to 0.57451, saving model to XXACT.best.hdf5

Epoch 00006: val_acc improved from 0.57451 to 0.59449, saving model to XXACT.best.hdf5

Epoch 00007: val_acc improved from 0.59449 to 0.63195, saving model to XXACT.best.hdf5

Epoch 00008: val_acc improved from 0.63195 to 0.64452, saving model to XXACT.best.hdf5

In [ ]:
results